In [12]:
import os
import shutil
import subprocess
from IPython.display import display, HTML
from tkinter import Tk, filedialog


In [13]:
temp_folder = 'tmp_frames'
result_folder = 'results'

if os.path.isdir(temp_folder):
    shutil.rmtree(temp_folder)
if os.path.isdir(result_folder):
    shutil.rmtree(result_folder)

os.mkdir(temp_folder)
os.mkdir(result_folder)


In [14]:
input_path = r'inputs\test.mp4'  

if not input_path:
    raise ValueError("No video selected!")

file_name = os.path.basename(input_path)
print(f"Selected Video: {file_name}")


Selected Video: test.mp4


In [15]:
print("Extracting frames from video...")

cmd = [
    'ffmpeg',
    '-i', input_path,
    '-qscale:v', '1',
    '-vsync', '0',
    f'{temp_folder}/frame_%08d.png'
]

process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

if process.returncode != 0:
    print(stderr.decode())
    raise RuntimeError("Error extracting frames")
else:
    frame_count = len(os.listdir(temp_folder))
    print(f"Extracted {frame_count} frames successfully!")


Extracting frames from video...
Extracted 356 frames successfully!


In [16]:
# Enhance the extracted frames using Real-ESRGAN
print("Enhancing frames using Real-ESRGAN...")

cmd = [
    'python', 'inference_realesrgan.py',
    '-n', 'RealESRGAN_x4plus',
    '-i', temp_folder,
    '--outscale', '4',
    '--face_enhance'
]

process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

if process.returncode != 0:
    print(stderr.decode())
    raise RuntimeError("Error enhancing frames")
else:
    print("Frame enhancement complete!")


Enhancing frames using Real-ESRGAN...
Frame enhancement complete!


In [17]:
import subprocess
import os

temp_folder = "tmp_frames"  
result_folder = "results"  
file_name = "video.mp4" 

# Set frames per second for the output video
fps = 15  

# Define output video path
output_video = os.path.join(result_folder, f"enhanced_{file_name}")

# Ensure the output folder exists
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

# Check if frames are in the temp folder
frame_files = os.listdir(temp_folder)
if not frame_files:
    raise FileNotFoundError("No frames found in the 'tmp_frames' folder.")

# Verify the naming convention of the frames (should match 'frame_00000001.png', etc.)
print(f"Frames found: {frame_files}")

# Define input frames path (ensure the frames have the correct naming pattern)
frame_input_path = os.path.join(temp_folder, 'frame_%08d.png')  

# Print message for debugging
print(f"Recreating video from enhanced frames at {frame_input_path}...")

# Run the ffmpeg command to create the video from frames
cmd = [
    'ffmpeg',
    '-i', frame_input_path,  
    '-c:v', 'libx264',  
    '-r', str(fps), 
    '-pix_fmt', 'yuv420p',  
    output_video 
]

# Run the subprocess to execute the command
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Check if the command ran successfully
if process.returncode != 0:
    print(f"Error recreating video: {stderr.decode()}")
    raise RuntimeError("Error recreating video")
else:
    print(f"Enhanced video saved as: {output_video}")


Frames found: ['frame_00000001.png', 'frame_00000002.png', 'frame_00000003.png', 'frame_00000004.png', 'frame_00000005.png', 'frame_00000006.png', 'frame_00000007.png', 'frame_00000008.png', 'frame_00000009.png', 'frame_00000010.png', 'frame_00000011.png', 'frame_00000012.png', 'frame_00000013.png', 'frame_00000014.png', 'frame_00000015.png', 'frame_00000016.png', 'frame_00000017.png', 'frame_00000018.png', 'frame_00000019.png', 'frame_00000020.png', 'frame_00000021.png', 'frame_00000022.png', 'frame_00000023.png', 'frame_00000024.png', 'frame_00000025.png', 'frame_00000026.png', 'frame_00000027.png', 'frame_00000028.png', 'frame_00000029.png', 'frame_00000030.png', 'frame_00000031.png', 'frame_00000032.png', 'frame_00000033.png', 'frame_00000034.png', 'frame_00000035.png', 'frame_00000036.png', 'frame_00000037.png', 'frame_00000038.png', 'frame_00000039.png', 'frame_00000040.png', 'frame_00000041.png', 'frame_00000042.png', 'frame_00000043.png', 'frame_00000044.png', 'frame_00000045.p

In [18]:
import cv2
import numpy as np

def compute_psnr(image1, image2):
    # Convert the images to grayscale (if needed)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

    # Compute Mean Squared Error (MSE)
    mse = np.mean((image1 - image2) ** 2)
    if mse == 0:
        return 100 

    # Compute PSNR
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def compare_videos(original_video_path, enhanced_video_path):
    original_video = cv2.VideoCapture(original_video_path)
    enhanced_video = cv2.VideoCapture(enhanced_video_path)

    psnr_values = []

    while original_video.isOpened() and enhanced_video.isOpened():
        ret1, frame1 = original_video.read()
        ret2, frame2 = enhanced_video.read()

        if not ret1 or not ret2:
            break

        psnr = compute_psnr(frame1, frame2)
        psnr_values.append(psnr)

    original_video.release()
    enhanced_video.release()

    avg_psnr = np.mean(psnr_values)
    print(f"Average PSNR: {avg_psnr:.2f} dB")

# Example usage
compare_videos('D:\\University\\IIT\\Level 7\\Final Year Project\\MVP\\super res\\yt vid enhc\\inputs\\test.mp4', 
               'D:\\University\\IIT\\Level 7\\Final Year Project\\MVP\\super res\\yt vid enhc\\results\\enhanced_video.mp4')


Average PSNR: 32.86 dB


In [20]:
import cv2
import numpy as np
import torch
import lpips
from skimage.metrics import structural_similarity as ssim

# Initialize LPIPS loss function
lpips_loss = lpips.LPIPS(net='alex')  # Uses AlexNet as the base network

def compute_psnr(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    if mse == 0:
        return 100  # No difference, perfect quality
    max_pixel = 255.0
    return 20 * np.log10(max_pixel / np.sqrt(mse))

def compute_ssim(image1, image2):
    return ssim(image1, image2, data_range=255, win_size=3, channel_axis=-1)  # FIXED

def compute_mse(image1, image2):
    return np.mean((image1 - image2) ** 2)

def compute_lpips(image1, image2):
    image1 = torch.from_numpy(image1).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    image2 = torch.from_numpy(image2).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    return lpips_loss(image1, image2).item()

def compare_videos(original_video_path, enhanced_video_path):
    original_video = cv2.VideoCapture(original_video_path)
    enhanced_video = cv2.VideoCapture(enhanced_video_path)

    psnr_values = []
    ssim_values = []
    mse_values = []
    lpips_values = []

    frame_count = 0

    while original_video.isOpened() and enhanced_video.isOpened():
        ret1, frame1 = original_video.read()
        ret2, frame2 = enhanced_video.read()

        if not ret1 or not ret2:
            break

        # Resize frames to the same size (you can choose any size here)
        frame1 = cv2.resize(frame1, (640, 640))  # Resize original frame
        frame2 = cv2.resize(frame2, (640, 640))  # Resize enhanced frame

        frame_count += 1
        print(f"Processing frame {frame_count}...")

        # Compute quality metrics
        psnr_values.append(compute_psnr(frame1, frame2))
        ssim_values.append(compute_ssim(frame1, frame2))
        mse_values.append(compute_mse(frame1, frame2))
        lpips_values.append(compute_lpips(frame1, frame2))

    original_video.release()
    enhanced_video.release()

    # Compute average values
    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    avg_mse = np.mean(mse_values)
    avg_lpips = np.mean(lpips_values)

    # Print results
    print("\n=== Video Quality Comparison Results ===")
    print(f"📌 Average PSNR:  {avg_psnr:.2f} dB (Higher is better)")
    print(f"📌 Average SSIM:  {avg_ssim:.4f} (1.0 = Perfect match)")
    print(f"📌 Average MSE:   {avg_mse:.2f} (Lower is better)")
    print(f"📌 Average LPIPS: {avg_lpips:.4f} (Lower is better, deep-learning based)")

# Example Usage
compare_videos(
    r"D:\University\IIT\Level 7\Final Year Project\MVP\super res\yt vid enhc\inputs\test.mp4",
    r"D:\University\IIT\Level 7\Final Year Project\MVP\super res\yt vid enhc\results\enhanced_video.mp4"
)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: c:\Users\thavi\AppData\Local\Programs\Python\Python310\lib\site-packages\lpips\weights\v0.1\alex.pth
Processing frame 1...
Processing frame 2...
Processing frame 3...
Processing frame 4...
Processing frame 5...
Processing frame 6...
Processing frame 7...
Processing frame 8...
Processing frame 9...
Processing frame 10...
Processing frame 11...
Processing frame 12...
Processing frame 13...
Processing frame 14...
Processing frame 15...
Processing frame 16...
Processing frame 17...
Processing frame 18...
Processing frame 19...
Processing frame 20...
Processing frame 21...
Processing frame 22...
Processing frame 23...
Processing frame 24...
Processing frame 25...
Processing frame 26...
Processing frame 27...
Processing frame 28...
Processing frame 29...
Processing frame 30...
Processing frame 31...
Processing frame 32...
Processing frame 33...
Processing frame 34...
Processing frame 35...
Processing 